First things first let's initialise your environment.

*Run the cell below to import your libraries and initialise your LUSID client*

In [1]:
# Import LUSID
import lusid.models as models
import lusid_sample_data as import_data

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
from dateutil.parser import parse
import pytz
import printer as prettyprint
import pandas as pd
import uuid
import math
import json

# Authenticate our user and create our API client
client = import_data.authenticate_secrets()

print ('LUSID Environment Initialised')
print ('API Version: ', client.metadata.get_lusid_versions().build_version)

LUSID Environment Initialised
API Version:  0.5.2643.0


![Initailise LUSID](img/multiplecurrencies-initialise.gif)

## 1) Set up your Scope

You will need to create a name for the scope that you will use for the fund. Read more about scopes in the [LUSID Knowledge Base: Scopes](https://support.lusid.com/what-is-a-scope-in-lusid-and-how-is-it-used).

*Run the cell below to create a name for your scope*

In [2]:
# Give the scope a unique identifier
scope_id = str(uuid.uuid4())[:4]
# Give the scope a descriptive name
scope_name = 'UK_High_Growth_Equities_Fund'
# Join the two together to get the full scope name
scope = '{}_{}'.format(scope_name, scope_id)
prettyprint.heading('Scope', scope)

Scope: UK_High_Growth_Equities_Fund_4c2e


![Scope](img/multiplecurrencies-scope.gif)

## 2) Load Instruments

Before you can take on any holdings or make any trades you need to ensure that your instrument universe has been populated. In this case you will import your instrument universe from a CSV file. Read more about instruments in LUSID in the [LUSID Knowledge Base: Instruments](https://support.lusid.com/what-is-an-instrument).

*Run the cell below to import your instrument universe*

In [3]:
# Import your instrument universe from a CSV file
instrument_universe = pd.read_csv('data/multiplecurrencies-instruments.csv')
# Look at the first 10 instruments
instrument_universe.head(n=10)

,instrument_name,client_internal,currency,isin,figi,exchange_code,country_issue,ticker,market_sector,security_type,coupon
0,Amazon_Nasdaq_AMZN,imd_34634534,USD,US0231351067,BBG000BVPXP1,UN,united_states_america,AMZN,equity,common_stock,NaN
1,Apple_Nasdaq_AAPL,imd_35345345,USD,US0378331005,BBG000B9XVV8,UN,united_states_america,AAPL,equity,common_stock,NaN
2,USTreasury_2.00_2021,imd_34535347,USD,US912828U816,BBG00FN3B5K8,BERLIN,united_states_america,T 2 12/31/21,govt,us_government,2.000
3,USTreasury_6.875_2025,imd_34534539,USD,US912810EV62,BBG000DQQNJ8,NEW YORK,united_states_america,T 6.875 08/15/25,govt,us_government,6.875
4,ExpressScripts_NYSE_ESRX,imd_34352311,USD,US30219G1085,BBG000C16621,UN,united_states_america,ESRX,equity,common_stock,NaN
5,TrinityIndustries_NYSE_TRN,imd_34235200,USD,US8965221091,BBG000BVL406,UN,united_states_america,TRN,equity,common_stock,NaN
6,Trex_NYSE_TREX,imd_32423956,USD,US89531P1057,BBG000BTGM43,UN,united_states_america,TREX,equity,common_stock,NaN
7,Cigna_NYSE_CI,imd_32452391,USD,US1255091092,BBG00KXXK940,UN,united_states_america,CI,equity,common_stock,NaN
8,Arcosa_NYSE_ACA,imd_23423409,USD,US0396531008,BBG00JGMWFQ5,UN,united_states_america,ACA,equity,common_stock,NaN


Now that you have the details for your instruments you can go ahead and create an instrument definition for each instrument. These can then be upserted into LUSID. Read about instrument definitions here [LUSID Knowledge Base: What is an Instrument?](https://support.lusid.com/what-is-an-instrument)

As part of this definition you will attach identifiers to your instruments. Read more about identifiers here [LUSID Knowledge Base: Which Instrument Identifier Schemes Should I Use With LUSID?](https://support.lusid.com/which-instrument-identifier-schemes-should-i-use-with-lusid)

You use an upsert method to add instrument definitions to the instrument universe in LUSID. Read more about the behaviour of the upsert method here [LUSID Knowledge Base: Upsert](https://support.lusid.com/upsert-command).

For further usage of the upsert instruments API call refer to the [LUSID API Docs: Upserting Instruments](https://docs.lusid.com/#operation/UpsertInstruments).

*Run the cell below to upsert your instruments into LUSID*

In [4]:
# Initialise a dictionary to hold your instrument definitions
instrument_definitions = {}

# Set the mapping between your identifier columns and the available LUSID identifiers
identifier_columns = {
    'isin': 'Isin',
    'figi': 'Figi',
    'ticker': 'Ticker',
    'client_internal': 'ClientInternal'
}

# Iterate over your instrument universe
for index, instrument in instrument_universe.iterrows():

    # Create your identifier dictionary for this instrument
    identifiers = {}
    for identifier_column, identifier_lusid in identifier_columns.items():
        identifiers[identifier_lusid] = models.InstrumentIdValue(
            value=instrument[identifier_column])
        
    # Create the definition for your instrument
    instrument_definitions[instrument['instrument_name']] = models.InstrumentDefinition(
        name=instrument['instrument_name'],
        identifiers=identifiers
    )

# Call LUSID to upsert your instrument defintions as a batch
response = client.instruments.upsert_instruments(
    requests=instrument_definitions)

# Pretty print the response from LUSID
prettyprint.instrument_response(response, identifier='Figi')

Instrument Successfully Upserted: Cigna_NYSE_CI
Figi ID: BBG00KXXK940
LUSID Instrument ID: LUID_0N8L1PEI


Instrument Successfully Upserted: ExpressScripts_NYSE_ESRX
Figi ID: BBG000C16621
LUSID Instrument ID: LUID_XPX95K5S


Instrument Successfully Upserted: USTreasury_6.875_2025
Figi ID: BBG000DQQNJ8
LUSID Instrument ID: LUID_J67SJTKE


Instrument Successfully Upserted: Apple_Nasdaq_AAPL
Figi ID: BBG000B9XVV8
LUSID Instrument ID: LUID_28QQHEDG


Instrument Successfully Upserted: Amazon_Nasdaq_AMZN
Figi ID: BBG000BVPXP1
LUSID Instrument ID: LUID_Q1JT811J


Instrument Successfully Upserted: Arcosa_NYSE_ACA
Figi ID: BBG00JGMWFQ5
LUSID Instrument ID: LUID_TML8WEH6


Instrument Successfully Upserted: Trex_NYSE_TREX
Figi ID: BBG000BTGM43
LUSID Instrument ID: LUID_HZI5VR1Z


Instrument Successfully Upserted: USTreasury_2.00_2021
Figi ID: BBG00FN3B5K8
LUSID Instrument ID: LUID_YCW2RZPU


Instrument Successfully Upserted: TrinityIndustries_NYSE_TRN
Figi ID: BBG000BVL406
LUSID Instrument ID: LU

![Instruments](img/multiplecurrencies-instruments.gif)

## 3) Create your Base Fund Portfolio

To create a portfolio you need to give it a name and code.

*Run the cell below to give your base fund portfolio a name and code*

In [5]:
# Give your portfolio a name & code
base_portfolio_name = '{}_base_fund'.format(scope_name)
base_portfolio_code = str(uuid.uuid4())
prettyprint.heading('Portfolio Name', base_portfolio_name)
prettyprint.heading('Portfolio Code', base_portfolio_code)

Portfolio Name: UK_High_Growth_Equities_Fund_base_fund
Portfolio Code: 26635eeb-a633-4460-bab9-fe4871701af3


Now that you have decided on the name and uniqu code for your portfolio you can create your base fund portfolio. Read more about portfolios in the [LUSID Knowledge Base: Portfolios](https://support.lusid.com/what-is-portfolio).

For further usage of the create portfolio API call refer to the [LUSID API Docs: Create Portfolio](https://docs.lusid.com/#operation/CreatePortfolio).

Note that when you create the portolio in the cell below you are creating it with a 'created' date of 365 days ago. This number is rather arbitary, in practice it should be the date the portfolio came into existence regardless of the system you first created it in, read more about the importance of the created date on a portfolio in the [LUSID Knowledge Base: Importance of Portfolio Creation Date](https://support.lusid.com/importance-of-portfolio-creation-date).

*Run the cell below to create your portfolio*

In [6]:
# The date your portfolio was first created
portfolio_creation_date = (datetime.now(pytz.UTC) - timedelta(days=365))

# Create the request to add your portfolio to LUSID
transaction_portfolio_request = models.CreateTransactionPortfolioRequest(
    display_name=base_portfolio_name,
    code=base_portfolio_code,
    base_currency='USD',
    description='The portfolio to hold our base fund',
    created=portfolio_creation_date)

# Call LUSID to create your portfolio
response = client.transaction_portfolios.create_portfolio(
    scope=scope,
    create_request=transaction_portfolio_request)

# Pretty print the response from LUSID
prettyprint.portfolio_response(response)

Portfolio Created
Scope: UK_High_Growth_Equities_Fund_4c2e
Code: 26635eeb-a633-4460-bab9-fe4871701af3
Portfolio Effective From: 2018-05-24 14:30:26.666435+00:00
Portfolio Created On: 2019-05-24 14:30:26.477332+00:00



![Base-Fund](img/multiplecurrencies-basefundportfolio.gif)

## 4) Set your Holdings

Now that you have your instrument universe populated you can load your initial holdings into your portfolio. In this case you will import your holdings from a CSV file.

*Run the cell below to import your take on balances*

In [7]:
#Import your holdings
holdings = pd.read_csv('data/multiplecurrencies-holdings.csv')
holdings.head()

,instrument_name,client_internal,isin,figi,quantity,unit_cost,total_cost,currency
0,Amazon_Nasdaq_AMZN,imd_34634534,US0231351067,BBG000BVPXP1,5000,1550.00,7750000.00,USD
1,Apple_Nasdaq_AAPL,imd_35345345,US0378331005,BBG000B9XVV8,49567,190.00,9417730.00,USD
2,USTreasury_2.00_2021,imd_34535347,US912828U816,BBG00FN3B5K8,121543,99.25,12063142.75,USD
3,USTreasury_6.875_2025,imd_34534539,US912810EV62,BBG000DQQNJ8,98444,140.98,13878635.12,USD


Now that you have imported your holdings you can add them to LUSID. You can do this by setting the holdings on a portfolio. Read more about how making an adjustment or setting the holdings on a portfolio affects it here [LUSID Knowledge Base: The Effect of Holding Adjustments](https://support.lusid.com/how-do-holding-adjustments-affect-a-portfolio).

For further usage of the set holdings API call refer to the [LUSID API Docs: Set Holdings](https://docs.lusid.com/#operation/SetHoldings).

*Run the cell below to upsert your holdings into LUSID*

In [8]:
# Initialise a list to hold your holding adjustments
holding_adjustments = []

# Set the effective date of these holdings to be 4 days ago
holdings_effective_date = datetime.now(pytz.UTC) - timedelta(days=4)

# Iterate over your holdings
for index, holding in holdings.iterrows():
    
    # Create a holding adjustment for this holding
    holding_adjustments.append(
        models.AdjustHoldingRequest(
            instrument_identifiers={
                    'Instrument/default/Figi': holding['figi']},
                tax_lots=[
                    models.TargetTaxLotRequest(
                        units=holding['quantity'],
                        cost=models.CurrencyAndAmount(
                            amount=holding['total_cost'],
                            currency=holding['currency']),
                        portfolio_cost=holding['total_cost'],
                        price=holding['unit_cost'])
                ]
        )
    )
    
# Call LUSID to set your holdings 
response = client.transaction_portfolios.set_holdings(
    scope=scope,
    code=base_portfolio_code,
    effective_at=holdings_effective_date,
    holding_adjustments=holding_adjustments)

# Pretty print the response 
prettyprint.set_holdings_response(response, scope, base_portfolio_name)

Holdings Successfully Set for Portfolio
Scope: UK_High_Growth_Equities_Fund_4c2e
Code: UK_High_Growth_Equities_Fund_base_fund
Holdings Effective From: 2019-05-20 14:30:27.216636+00:00
Holdings Created On: 2019-05-24 14:30:27.392495+00:00



![Base-Fund-Hodlings](img/multiplecurrencies-basefundportfolioholdings.gif)

## 5) Add Corporate Actions

In [9]:
"""
corporate_action_source_id = '{}_corporate_action_source'.format(base_portfolio_name)

corporate_action_request = models.CreateCorporateActionSourceRequest(
    scope=scope,
    code=corporate_action_source_id)

response = client.corporate_action_sources.create_corporate_action_source(
    request=corporate_action_request)
    
prettyprint.corporate_action_response(response)
"""

"\ncorporate_action_source_id = '{}_corporate_action_source'.format(base_portfolio_name)\n\ncorporate_action_request = models.CreateCorporateActionSourceRequest(\n    scope=scope,\n    code=corporate_action_source_id)\n\nresponse = client.corporate_action_sources.create_corporate_action_source(\n    request=corporate_action_request)\n    \nprettyprint.corporate_action_response(response)\n"

![Corporate-Action-Source](img/multiplecurrencies-corporateactionsource.gif)

In [10]:
corporate_actions = pd.read_csv('data/corporateactions.csv')
corporate_actions.head()

,code,action_description,description,announcement_date,ex_date,record_date,payment_date,input_instrument_figi,instrument_input_ticker,input_instrument_name,input_units_factor,input_cost_factor,output_instrument_figi,output_instrument_name,output_ticker,output_instrument_internal,output_units_factor,output_cost_factor,dividend_yield
0,5943592342,dividend,Apple Dividend,29/01/2019,08/02/2019,11/02/2019,14/02/2019,BBG000B9XVV8,AAPL,APPLE INC,1,1,NaN,NaN,NaN,CCY_GBP,0.0170,0.0000,0.017
1,5943592343,merger,Cigna & Express Scripts Merger,08/03/2018,10/12/2018,13/12/2018,20/12/2018,BBG000C16621,ESRX,EXPRESS SCRIPTS HOLDINGS CO,1,1,NaN,NaN,NaN,CCY_USD,48.7500,0.0000,NaN
2,5943592343,merger,Cigna & Express Scripts Merger,08/03/2018,10/12/2018,13/12/2018,20/12/2018,BBG000C16621,ESRX,EXPRESS SCRIPTS HOLDINGS CO,1,1,BBG00KXXK940,CIGNA CORP,CI,NaN,0.2434,1.9337,NaN
3,5943592347,spin-off,Acosa spin-off from Trinity Industries,12/12/2017,17/10/2018,20/10/2018,01/11/2018,BBG000BVL406,TRN,TRINITY INDUSTRIES INC,3,1,BBG00JGMWFQ5,ARCOSA INC,ACA,NaN,1.0000,1.0324,NaN
4,5943592498,split,Trex 2 for 1 stock split,07/05/2018,21/05/2018,23/05/2018,19/06/2018,BBG000BTGM43,TREX,TREX COMPANY INC,1,1,BBG000BTGM43,TREX COMPANY INC,TREX,NaN,2.0000,2.0000,NaN


In [11]:
"""
actions = {}


for row, action in corporate_actions.iterrows():
    
    actions[action['code']] = {}
    
    actions[action['code']].setdefault(
        'input_transition', models.CorporateActionTransitionComponentRequest(
            instrument_identifiers={
                'Instrument/default/Figi': action['input_instrument_figi']
            },
            units_factor=action['input_units_factor'],
            cost_factor=action['input_cost_factor']))
    
    actions[action['code']].setdefault(
        'output_transitions', []).append(
            models.CorporateActionTransitionComponentRequest(
                instrument_identifiers={
                    'Instrument/default/Figi': action['input_instrument_figi']
                },
                units_factor=action['input_units_factor'],
                cost_factor=action['input_cost_factor']))
    
actions_list = []

for action_id, action in actions.items():
    
    transition = models.CorporateActionTransitionRequest(
        input_transition=actions[action_id]['input_transition'],
        output_transitions=actions[action_id]['output_transitions']
    )
    
    actions_list.append(
        models.CreateCorporateAction(
            corporate_action_code=action_id,
            announcement_date=datetime.now(pytz.UTC)+timedelta(days=1),
            ex_date=datetime.now(pytz.UTC)+timedelta(days=1),
            record_date=datetime.now(pytz.UTC)+timedelta(days=1),
            payment_date=datetime.now(pytz.UTC)+timedelta(days=1),
            transitions=[transition])
    )

response = client.corporate_action_sources.batch_upsert_corporate_actions(
    scope=scope,
    code=corporate_action_source_id,
    actions=actions_list)

prettyprint.corporate_actions_added_response(response)
"""

"\nactions = {}\n\n\nfor row, action in corporate_actions.iterrows():\n    \n    actions[action['code']] = {}\n    \n    actions[action['code']].setdefault(\n        'input_transition', models.CorporateActionTransitionComponentRequest(\n            instrument_identifiers={\n                'Instrument/default/Figi': action['input_instrument_figi']\n            },\n            units_factor=action['input_units_factor'],\n            cost_factor=action['input_cost_factor']))\n    \n    actions[action['code']].setdefault(\n        'output_transitions', []).append(\n            models.CorporateActionTransitionComponentRequest(\n                instrument_identifiers={\n                    'Instrument/default/Figi': action['input_instrument_figi']\n                },\n                units_factor=action['input_units_factor'],\n                cost_factor=action['input_cost_factor']))\n    \nactions_list = []\n\nfor action_id, action in actions.items():\n    \n    transition = models.Corporat

In [12]:
"""
response = client.transaction_portfolios.upsert_portfolio_details(
    scope=scope,
    code=base_portfolio_name,
    create_portfolio_details=models.CreatePortfolioDetails(
        corporate_action_source_id=models.ResourceId(
            scope=scope,
            code=corporate_action_source_id),
        base_currency='GBP'))

prettyprint.portfolio_details_response(response)
"""

"\nresponse = client.transaction_portfolios.upsert_portfolio_details(\n    scope=scope,\n    code=base_portfolio_name,\n    create_portfolio_details=models.CreatePortfolioDetails(\n        corporate_action_source_id=models.ResourceId(\n            scope=scope,\n            code=corporate_action_source_id),\n        base_currency='GBP'))\n\nprettyprint.portfolio_details_response(response)\n"

![Corporate-Actions](img/multiplecurrencies-corporateactions.gif)

## 6) Value our Base Fund Portfolio

### a) Upsert Market Quotes

To value a portfolio in LUSID you need to upsert market data quotes against the underlying holdings or specify an analytics library to use. Read more about aggregating and valuing portfolios in the [LUSID Knowledge Base: Aggregations and Valuations](https://support.lusid.com/what-is-a-valuation). 

In this case you will upsert market data quotes to the quote store to be used in an aggregation request. You will import these quotes from a CSV file.

*Run the cell below to import the market data prices*

In [13]:
# Import market data for the day that the holdings of the fund are effective from
prices = pd.read_csv('data/multiplecurrencies-prices.csv')
prices.head()

,price,type,currency,instrument_name,instrument_figi,instrument_internal
0,1622.65,close,USD,Amazon_Nasdaq_AMZN,BBG000BVPXP1,imd_34634534
1,170.80,close,USD,Apple_Nasdaq_AAPL,BBG000B9XVV8,imd_34535347
2,98.00,close,USD,USTreasury_2.00_2021,BBG00FN3B5K8,imd_34535347
3,142.36,close,USD,USTreasury_6.875_2025,BBG000DQQNJ8,imd_34534539


Now that you have imported the market data you can add it to the quote store in LUSID. Read more about what a quote is in the [LUSID Knowledge Base: What is a Quote?](https://support.lusid.com/what-is-a-quote).

For further usage of the Upsert Quotes API call refer to the [LUSID API Docs: Upsert Quotes](https://docs.lusid.com/#operation/UpsertQuotes).

*Run the cell below to upsert the market data quotes into LUSID*

In [14]:
# Initialise an empty instrument quotes list to hold the quotes
instrument_quotes = []

# Iterate over the quotes
for index, quote in prices.iterrows():
    
    # Add the quote to the list 
    instrument_quotes.append(
        models.UpsertQuoteRequest(
            quote_id=models.QuoteId(
                provider='DataScope',
                price_source='USDRC',
                instrument_id=quote['instrument_figi'],
                instrument_id_type='Figi',
                quote_type='Price',
                price_side='Mid'),
            metric_value=models.MetricValue(
                value=quote['price'],
                unit=quote['currency']),
            effective_at=holdings_effective_date,
            lineage='InternalSystem'
            )
    )

# Upsert the quotes into LUSID
response = client.quotes.upsert_quotes(
    scope=scope,
    quotes=instrument_quotes)

# Pretty print the response
prettyprint.upsert_quotes_response(response)

Quotes Successfully Upserted At: 2019-05-24 14:30:27.976156+00:00


![Analytic-Stores](img/multiplecurrencies-quotestore.gif)

### b) Value using Quotes via Aggregation

Now that the quotes have been added to the quote store you can aggregate your base fund portfolio. The logic for an aggregation is controled by a LUSID recipe. Read more about recipes in the [LUSID Knowledge Base: What is a Recipe and How Are They Used?](https://support.lusid.com/what-is-a-recipe-and-how-are-they-used).

For further usage of the Get Aggregation by Portfolio API call refer to the [LUSID API Docs: Get Aggregation by Portfolio](https://docs.lusid.com/#operation/GetAggregationByPortfolio).

*Run the cell below to aggregate and value the base fund*

In [15]:
# Specify the inline recipe used to perform the aggregation
inline_recipe = models.ConfigurationRecipe(
    code='quotes_recipe',
    market=models.MarketContext(
        market_rules=[
            models.MarketDataKeyRule(
               key='Equity.Figi.*',
               supplier='DataScope',
               data_scope=scope,
               quote_type='Price',
               price_side='Mid'),
           models.MarketDataKeyRule(
               key='Equity.LusidInstrumentId.*',
               supplier='DataScope',
               data_scope=scope,
               quote_type='Price',
               price_side='Mid'),
            models.MarketDataKeyRule(
               key='Fx.CurrencyPair.*',
               supplier='DataScope',
               data_scope=scope,
               quote_type='Rate',
               price_side='Mid')
        ],
        suppliers=models.MarketContextSuppliers(
            commodity='DataScope',
            credit='DataScope',
            equity='DataScope',
            fx='DataScope',
            rates='DataScope'),
        options=models.MarketOptions(
            default_supplier='DataScope',
            default_instrument_code_type='Figi',
            default_scope=scope)
    )
)

# Create the aggregation request
aggregation_request = models.AggregationRequest(
    inline_recipe=inline_recipe,
    effective_at=holdings_effective_date,
    metrics=[
        models.AggregateSpec(key='Holding/default/SubHoldingKey',
        op='Value'),
        models.AggregateSpec(key='Instrument/default/Name',
        op='Value'),
        models.AggregateSpec(key='Holding/default/Units',
        op='Sum'),
        models.AggregateSpec(key='Holding/default/Cost',
        op='Sum'),
        models.AggregateSpec(key='Holding/default/PV',
        op='Sum'),
    ],
    group_by=[
        'Holding/default/SubHoldingKey'
    ])

# Call LUSID to perform the aggregation
response = client.aggregation.get_aggregation_by_portfolio(
    scope=scope,
    code=base_portfolio_code,
    request=aggregation_request)

# Pretty print the response
prettyprint.aggregation_response_generic(response)

Aggregation Results: 
Holding/default/SubHoldingKey: LusidInstrumentId=LUID_Q1JT811J/USD
Instrument/default/Name: Amazon_Nasdaq_AMZN
Sum(Holding/default/Units): 5000.0
Sum(Holding/default/Cost): 7750000.0
Sum(Holding/default/PV): 8113250.0


Holding/default/SubHoldingKey: LusidInstrumentId=LUID_28QQHEDG/USD
Instrument/default/Name: Apple_Nasdaq_AAPL
Sum(Holding/default/Units): 49567.0
Sum(Holding/default/Cost): 9417730.0
Sum(Holding/default/PV): 8466043.6


Holding/default/SubHoldingKey: LusidInstrumentId=LUID_YCW2RZPU/USD
Instrument/default/Name: USTreasury_2.00_2021
Sum(Holding/default/Units): 121543.0
Sum(Holding/default/Cost): 12063142.75
Sum(Holding/default/PV): 11911214.0


Holding/default/SubHoldingKey: LusidInstrumentId=LUID_J67SJTKE/USD
Instrument/default/Name: USTreasury_6.875_2025
Sum(Holding/default/Units): 98444.0
Sum(Holding/default/Cost): 13878635.12
Sum(Holding/default/PV): 14014487.84




![Analytic-Stores](img/multiplecurrencies-aggregatedbase.gif)

## 7) Securitise your Base Fund Portfolio

### a) Securitise with x units

Now that you have created and valued your base fund, you need to securitise it so that it can be held by other portfolios. Before you create the instrument that allows other portfolios to hold the base fund you need to create a property to hold the total number of units of the fund available to be held.

To do this you can make use of a property. Read more about properties in the [LUSID Knowledge Base: What is a Property?](https://support.lusid.com/what-is-a-property).

For further usage of the Create Property Definition API call refer to the [LUSID API Docs: Create Property Definition](https://docs.lusid.com/#operation/CreatePropertyDefinition).

*Run the cell below to create a property to hold the total circulation of your base fund*

In [16]:
# Create your request to define a new property
property_request = models.CreatePropertyDefinitionRequest(
    domain='Instrument',
    scope=scope,
    code='total_circulation',
    value_required=False,
    display_name='total_circulation',
    data_type_id=models.ResourceId(scope='default', code='number'))

# Call LUSID to create uour new property
response = client.property_definitions.create_property_definition(
    definition=property_request)

# Grab the key off the response to use when referencing this property in other LUSID calls
circulation_property_key = response.key

# Pretty print your key
prettyprint.heading('Circulation Property Key', circulation_property_key)

Circulation Property Key: Instrument/UK_High_Growth_Equities_Fund_4c2e/total_circulation


![Analytic-Stores](img/multiplecurrencies-circulationproperty.gif)

You can now create an securitise the base fund portfolio as an instrument, allowing it to be held by other portfolios.

*Run the cell below to securitise the base fund*

In [17]:
# Create the instrument definition for the base fund
instrument_definition = models.InstrumentDefinition(
    name=base_portfolio_name,
    identifiers={'ClientInternal': models.InstrumentIdValue(
        value=base_portfolio_code)},
    properties=[
        models.InstrumentProperty(
            key=circulation_property_key,
            value=models.PropertyValue(
                metric_value=models.MetricValue(
                    value=40000.00)
            )
        )
    ],
    look_through_portfolio_id=models.ResourceId(
        scope=scope,
        code=base_portfolio_code)
)

# Call LUSID to upser the instrument
response = client.instruments.upsert_instruments(
    requests={
        'securitise-base-fund':instrument_definition
    }
)

# Pretty print the response
prettyprint.instrument_response(response, identifier='ClientInternal')

Instrument Successfully Upserted: securitise-base-fund
ClientInternal ID: 26635eeb-a633-4460-bab9-fe4871701af3
LUSID Instrument ID: LUID_R99W1V23
Lookthrough Portfolio Scope: UK_High_Growth_Equities_Fund_4c2e
Lookthrough Portfolio Code 26635eeb-a633-4460-bab9-fe4871701af3


1  instruments upserted successfully
0  instrument upsert failures


![Analytic-Stores](img/multiplecurrencies-securitisebase.gif)

### b) Calculalate price from aggregation and store in the quote store

So that you can accurately value any portfolios that hold the newly created base fund instrument, you need to store a price in the quotes store against it. To do this you can re-value the base fund portfolio as you did earlier. The only difference is that in this case you are grouping all the holdings together to get the total value of the fund.

*Run the cell below to get the total value of the fund*

In [18]:
# Create the aggregation request, using the recipe defined earlier
aggregation_request = models.AggregationRequest(
    inline_recipe=inline_recipe,
    effective_at=holdings_effective_date,
    metrics=[
        models.AggregateSpec(key='Holding/default/Cost',
        op='Sum'),
        models.AggregateSpec(key='Holding/default/PV',
        op='Sum'),
        models.AggregateSpec(key='Portfolio/default/Name',
        op='Value'),
    ],
    group_by=[
        'Portfolio/default/Name'
    ])

# Call LUSID to aggregate and value the base fund
response = client.aggregation.get_aggregation_by_portfolio(
    scope=scope,
    code=base_portfolio_code,
    request=aggregation_request)

# Pretty print the result
prettyprint.aggregation_response_generic(response)

# Set the total value and total cost of the base fund when it was first created
base_fund_total_value = response.data[0]['Sum(Holding/default/PV)']
base_fund_total_cost = response.data[0]['Sum(Holding/default/Cost)']

Aggregation Results: 
Sum(Holding/default/Cost): 43109507.87
Sum(Holding/default/PV): 42504995.44
Portfolio/default/Name: UK_High_Growth_Equities_Fund_base_fund




Now that you have the total value of the fund you need to divide this by its total circulation to get the unit price of the base fund. 

*Run the cell below to calculate the unit price of the base fund and upsert this quote into the quotes store*

In [19]:

# Get the base fund instrument
base_fund_instrument = client.instruments.get_instrument(
    identifier_type='ClientInternal',
    identifier=base_portfolio_code,
    instrument_property_keys=[circulation_property_key])

# Get the unique Lusid Instrument Id from the base fund instrument
luid = base_fund_instrument.lusid_instrument_id
# Get the total circulation from the base fund instrument
base_fund_total_circulation = round(base_fund_instrument.properties[0].value, 0)
# Calculate the unit price of the base fund from the total value and the total circulation
base_fund_unit_price = round(base_fund_total_value / base_fund_total_circulation, 2)

# Create a quote for the unit price of the base fund
instrument_quote = models.UpsertQuoteRequest(
    quote_id=models.QuoteId(
        provider='DataScope',
        price_source='USDRC',
        instrument_id=luid,
        instrument_id_type='LusidInstrumentId',
        quote_type='Price',
        price_side='Mid'),
    metric_value=models.MetricValue(
        value=base_fund_unit_price,
        unit='USD'),
    effective_at=holdings_effective_date,
    lineage='InternalSystem'
)

# Call LUSID to upsert the quote
response = client.quotes.upsert_quotes(
    scope=scope,
    quotes=[instrument_quote])

# Pretty print the total value, circulation and unit price of the fund
prettyprint.heading('Base Fund Total Circulation', str(base_fund_total_circulation))
prettyprint.heading('Base Fund Total Value', str(base_fund_total_value))
prettyprint.heading('Base Fund Unit Price', str(base_fund_unit_price))
# Pretty print the response from LUSID

prettyprint.upsert_quotes_response(response)

Base Fund Total Circulation: 40000.0
Base Fund Total Value: 42504995.44
Base Fund Unit Price: 1062.62


'\nprettyprint.upsert_quotes_response(response)\n'

![Base-Fund-Price](img/multiplecurrencies-basefundprice.gif)

## 8) Create Foreign Currency Portfolios

Now that you've succesfully securitised the base fund and associated it with a unit price, you can create additional portfolios to hold the fund. In this case you want to offer the fund in five different currencies.

Note that each portfolio is created with the appropriate base currency.

*Run the cell below to create a portfolio for each currency you'd like to offer the fund in*

In [20]:
# Define the 5 fund currencies to offer the fund in
fund_currencies = {
    'AUD': None,
    'GBP': None,
    'USD': None,
    'EUR': None,
    'JPY': None
}

# Iterate over each currency
for currency in fund_currencies.keys():
    
    # Create the portfolio name & code for this currency
    portfolio_name = '{}_{}'.format(base_portfolio_name, currency)
    portfolio_code = str(uuid.uuid4())
    fund_currencies[currency] = portfolio_code
    
    # Build the request to create your portfolio in this currency
    transaction_portfolio_request = models.CreateTransactionPortfolioRequest(
        display_name=portfolio_name,
        code=portfolio_code,
        base_currency=currency,
        description='The portfolio to hold our base fund in currency {}'.format(currency),
        created=portfolio_creation_date)

    # Call LUSID to create your portfolio
    portfolio_response = client.transaction_portfolios.create_portfolio(
        scope=scope,
        create_request=transaction_portfolio_request)

    # Pretty print the response from LUSID
    prettyprint.portfolio_response(portfolio_response)

Portfolio Created
Scope: UK_High_Growth_Equities_Fund_4c2e
Code: cb68d80c-8cbc-4efc-8199-fd1544e98aa8
Portfolio Effective From: 2018-05-24 14:30:26.666435+00:00
Portfolio Created On: 2019-05-24 14:30:31.945946+00:00

Portfolio Created
Scope: UK_High_Growth_Equities_Fund_4c2e
Code: 7aead1d3-69a9-41f4-8d56-7dcf47b0ca24
Portfolio Effective From: 2018-05-24 14:30:26.666435+00:00
Portfolio Created On: 2019-05-24 14:30:32.471448+00:00

Portfolio Created
Scope: UK_High_Growth_Equities_Fund_4c2e
Code: a21580e9-fb2e-4a58-b7bd-953f48f2dc65
Portfolio Effective From: 2018-05-24 14:30:26.666435+00:00
Portfolio Created On: 2019-05-24 14:30:32.872024+00:00

Portfolio Created
Scope: UK_High_Growth_Equities_Fund_4c2e
Code: 849bfece-e763-470d-bcd0-031a08f1d70e
Portfolio Effective From: 2018-05-24 14:30:26.666435+00:00
Portfolio Created On: 2019-05-24 14:30:33.292462+00:00

Portfolio Created
Scope: UK_High_Growth_Equities_Fund_4c2e
Code: ee9cc9a4-cbe0-4c9b-b179-057db13ceca9
Portfolio Effective From: 2018

![Foreign-Currency-Portfolios](img/multiplecurrencies-foreigncurrencyportfolios.gif)

## 9) Add Base Fund Units

Now that the portfolios have been created, you can add units of the base fund to them.

To do this you need to know the exchange rate at the time the currency portfolios take on the base fund units. This allows you to calculate the portfolio cost of the holdings.

In this case you will import the foreign exchange rates from a CSV file.

*Run the cell below to import the foreign exchange rates*

In [21]:
# Import the FX rates
fx_rates = pd.read_csv('data/multiplecurrencies-fxrates.csv')
fx_rates.head()

,pair,rate
0,GBP/USD,1.310
1,AUD/USD,0.720
2,EUR/USD,1.140
3,JPY/USD,0.009


Now that you have imported the FX rates you can upsert them into the quote store.

*Run the cell below to upsert the FX rates into the quote store*

In [22]:
# Prepare an empty list to hold the quote requests
quote_requests = []

# Iterate over each FX rate
for index, fx_rate in fx_rates.iterrows():
    
    # Add a quote for the FX rate
    quote_requests.append(
        models.UpsertQuoteRequest(
            quote_id=models.QuoteId(
                provider='DataScope',
                price_source='USDRC',
                instrument_id=fx_rate['pair'],
                instrument_id_type='CurrencyPair',
                quote_type='Rate',
                price_side='Mid'),
            metric_value=models.MetricValue(
                value=fx_rate['rate'],
                unit='rate'),
            effective_at=holdings_effective_date,
            lineage='InternalSystem'))

    # Create the reverse pair name 
    reverse_pair = '/'.join(fx_rate['pair'].split('/')[::-1])

    # Add a quote for the reverse FX rate
    quote_requests.append(
        models.UpsertQuoteRequest(
            quote_id=models.QuoteId(
                provider='DataScope',
                price_source='USDRC',
                instrument_id=reverse_pair,
                instrument_id_type='CurrencyPair',
                quote_type='Rate',
                price_side='Mid'),
            metric_value=models.MetricValue(
                value=1/fx_rate['rate'],
                unit='rate'),
            effective_at=holdings_effective_date,
            lineage='InternalSystem'))

# Upsert the quotes into LUSID
response = client.quotes.upsert_quotes(
    scope=scope,
    quotes=quote_requests)

# Pretty print the response
prettyprint.upsert_quotes_response(response)

Quotes Successfully Upserted At: 2019-05-24 14:30:34.123897+00:00


Now that you have loaded the FX rates into the quote store you can add units of the securitised base fund to each of the currency portfolios. In this case you will give each currency portfolio the same amount of units.

Note that here you are pulling the FX rates back out of the quote store using the Get Quotes API call. For further usage of the Get Quotes API call refer to the [LUSID API Docs: Get Quotes](https://docs.lusid.com/#operation/GetQuotes).

For further usage of the Adjust Holdings API call refer to the [LUSID API Docs: Adjust Holdings](https://docs.lusid.com/#operation/AdjustHoldings).

*Run the cell below to adjust the holdings of the currency portfolios to hold the base fund*

In [23]:
# Work out the base fund units to give to each portfolio, dividing equally amongst them
base_fund_units_per_portfolio = base_fund_total_circulation/len(fund_currencies)
# Work out the total cost for each portfolio in the currency of the base fund (USD)
total_cost_per_portfolio = base_fund_total_cost/len(fund_currencies)

# Iteratve over each currency
for currency, currency_portfolio_code in fund_currencies.items():
    
    # If the currency is the same as the base fund, skip getting an FX rate
    if currency!='USD':
        # Get the appropriate FX rate from the quotes store
        fx_rate = client.quotes.get_quotes(
            scope=scope,
            quote_ids=[
                models.QuoteId(
                    provider='DataScope',
                    price_source='USDRC',
                    instrument_id='{}/USD'.format(currency),
                    instrument_id_type='CurrencyPair',
                    quote_type='Rate',
                    price_side='Mid')
            ],
            effective_at=holdings_effective_date).found[0].metric_value.value
    else:
        # Use an FX rate of 1 if the currency is the same as the base fund
        fx_rate = 1
    
    # Initialise an empty list for the holding adjustments
    holding_adjustments = []
    
    # Add a holding adjustment to the list for this currency portfolio
    holding_adjustments.append(
        models.AdjustHoldingRequest(
            instrument_identifiers={
                    'Instrument/default/ClientInternal': base_portfolio_code},
                tax_lots=[
                    models.TargetTaxLotRequest(
                        units=base_fund_units_per_portfolio,
                        cost=models.CurrencyAndAmount(
                            amount=total_cost_per_portfolio,
                            currency='USD'),
                        portfolio_cost=total_cost_per_portfolio/fx_rate, 
                        price=base_fund_total_cost/base_fund_total_circulation)
                ]
        )
    )

    # Call LUSID to adjust your holdings
    response = client.transaction_portfolios.adjust_holdings(
        scope=scope,
        code=currency_portfolio_code,
        effective_at=holdings_effective_date,
        holding_adjustments=holding_adjustments)
    
    # Pretty print the response from LUSID
    prettyprint.set_holdings_response(response, scope, currency_portfolio_code)

Holdings Successfully Set for Portfolio
Scope: UK_High_Growth_Equities_Fund_4c2e
Code: cb68d80c-8cbc-4efc-8199-fd1544e98aa8
Holdings Effective From: 2019-05-20 14:30:27.216636+00:00
Holdings Created On: 2019-05-24 14:30:34.853154+00:00

Holdings Successfully Set for Portfolio
Scope: UK_High_Growth_Equities_Fund_4c2e
Code: 7aead1d3-69a9-41f4-8d56-7dcf47b0ca24
Holdings Effective From: 2019-05-20 14:30:27.216636+00:00
Holdings Created On: 2019-05-24 14:30:35.622271+00:00

Holdings Successfully Set for Portfolio
Scope: UK_High_Growth_Equities_Fund_4c2e
Code: a21580e9-fb2e-4a58-b7bd-953f48f2dc65
Holdings Effective From: 2019-05-20 14:30:27.216636+00:00
Holdings Created On: 2019-05-24 14:30:36.519933+00:00

Holdings Successfully Set for Portfolio
Scope: UK_High_Growth_Equities_Fund_4c2e
Code: 849bfece-e763-470d-bcd0-031a08f1d70e
Holdings Effective From: 2019-05-20 14:30:27.216636+00:00
Holdings Created On: 2019-05-24 14:30:37.298395+00:00

Holdings Successfully Set for Portfolio
Scope: UK_Hi

![Foreign-Currency-Portfolios](img/multiplecurrencies-foreigncurrencyportfoliosholdbase.gif)

## 10) Add Hedging Transactions

Even though you are offering your base fund in different currencies, you don't want fluctuations in the exchange rate to affect the performance of each currency portfolio. You therefore plan to hedge against the foreign exchange rate risk. Each hedge comes in the form of a forward fx sell contract which you plan to roll over weekly. 

Read more about creating bespoke instruments in the [LUSID Knowledge Base: Creating Bespoke Instruments](https://support.lusid.com/creating-bespoke-instruments). 

*Run the cell below to create your first hedging contracts*

In [24]:
# Initialise a dictionary to hold the hedging contracts
hedge_contracts = {}

# Iterate over each currency
for currency in fund_currencies.keys():
    
    # If the currency matches the base fund there is no need for a hedge
    if currency == 'USD':
        continue
        
    # Get the current foreign exchange rate for the first hedge contract
    fx_rate = client.quotes.get_quotes(
        scope=scope,
        quote_ids=[
            models.QuoteId(
                provider='DataScope',
                price_source='USDRC',
                instrument_id='{}/USD'.format(currency),
                instrument_id_type='CurrencyPair',
                quote_type='Rate',
                price_side='Mid')
        ],
        effective_at=holdings_effective_date).found[0].metric_value.value

    # Create the bespoke hedge contract definition
    hedge_contract = {
        "contract_id": "tid_{}".format(uuid.uuid4()),
        "type": "FwdFxSell",
        "legs": 2,
        "transaction_date": holdings_effective_date.isoformat(),
        "settlement_date": (holdings_effective_date+timedelta(days=7)).isoformat(),
        "strike_price": 1/fx_rate,
        "leg1": {
            "type": "sell",
            "currency": "USD",
            "amount": round(total_cost_per_portfolio, 0)
        },
        "leg2": {
            "type": "buy",
            "currency": currency,
            "amount": round(total_cost_per_portfolio, 0) / fx_rate
        }
    }

     # Create the definition for your instrument, attaching the bespoke contract
    hedge_contracts['{}_hedge'.format(currency)] = models.InstrumentDefinition(
        name='hedge_{}'.format(currency),
        identifiers={
            'ClientInternal': models.InstrumentIdValue(
                value=hedge_contract["contract_id"])},
        definition=models.InstrumentEconomicDefinition(
            instrument_format='JSON',
            content=json.dumps(hedge_contract)
        )
    )
        
# Call LUSID to upsert your instrument defintions
response = client.instruments.upsert_instruments(
    requests=hedge_contracts)

# Pretty print the response from LUSID
prettyprint.instrument_response(response)

Instrument Successfully Upserted: EUR_hedge
ClientInternal ID: tid_d178ca5e-737a-4d4b-937d-0861efe47341
LUSID Instrument ID: LUID_D2G221EI


Instrument Successfully Upserted: AUD_hedge
ClientInternal ID: tid_d90c1ed2-5cc3-4cfd-b18e-61e18530d1d8
LUSID Instrument ID: LUID_A16AS0XM


Instrument Successfully Upserted: JPY_hedge
ClientInternal ID: tid_9b0465a0-b399-4be7-970b-8f14094bca19
LUSID Instrument ID: LUID_42V89JPH


Instrument Successfully Upserted: GBP_hedge
ClientInternal ID: tid_1788bd9e-6cbd-435b-a9fc-190d974d73e8
LUSID Instrument ID: LUID_HYLAGLRF


4  instruments upserted successfully
0  instrument upsert failures


Now that you've added your hedge contracts to your instrument master inside LUSID you can log transactions against them. Read more about transactions in the [LUSID Knowledge Base: What is a Transaction?](https://support.lusid.com/what-is-a-transaction). 

*Run the cell below to upsert a hedging transaction for each currency portfolio*

In [25]:
# Iterate over the currencies
for currency, currency_portfolio_code in fund_currencies.items():
    
    if currency == 'USD':
        continue
        
    # Initialse a list to hold the hedging transaction
    hedge_transactions = []

    # Retrieve the bespoke hedge contract from the hedging contract definition
    contract = json.loads(client.instruments.get_instrument(
        identifier_type='ClientInternal',
        identifier=hedge_contracts['{}_hedge'.format(currency)].identifiers['ClientInternal'].value
    ).instrument_definition.content)

    # Build and add the hedge transaction for this currency to the list of transactions
    hedge_transactions.append(
        models.TransactionRequest(
            transaction_id=contract['contract_id'],
            type='Buy',
            instrument_identifiers={
                'Instrument/default/ClientInternal': contract['contract_id']},
            transaction_date='T'.join(contract['transaction_date'].split(' ')),
            settlement_date='T'.join(contract['settlement_date'].split(' ')),
            units=1,
            transaction_price=models.TransactionPrice(
                price=0,
                type='Price'),
            total_consideration=models.CurrencyAndAmount(
                amount=0,
                currency=currency),
            transaction_currency='USD',
            source='Client',
        )
    )
    
    # Upsert the transaction into LUSID
    response = client.transaction_portfolios.upsert_transactions(
        scope=scope,
        code=currency_portfolio_code,
        transactions=hedge_transactions
        )
  
    # Pretty print the response from LUSID
    prettyprint.transactions_response(
        response, 
        scope, 
        currency_portfolio_code)

Transactions Successfully Upserted into Portfolio
Scope: UK_High_Growth_Equities_Fund_4c2e
Code: cb68d80c-8cbc-4efc-8199-fd1544e98aa8
Transactions Effective From: 2019-05-20 14:30:27.216636+00:00
Transactions Created On: 2019-05-24 14:30:39.676683+00:00

Transactions Successfully Upserted into Portfolio
Scope: UK_High_Growth_Equities_Fund_4c2e
Code: 7aead1d3-69a9-41f4-8d56-7dcf47b0ca24
Transactions Effective From: 2019-05-20 14:30:27.216636+00:00
Transactions Created On: 2019-05-24 14:30:40.504107+00:00

Transactions Successfully Upserted into Portfolio
Scope: UK_High_Growth_Equities_Fund_4c2e
Code: 849bfece-e763-470d-bcd0-031a08f1d70e
Transactions Effective From: 2019-05-20 14:30:27.216636+00:00
Transactions Created On: 2019-05-24 14:30:41.273506+00:00

Transactions Successfully Upserted into Portfolio
Scope: UK_High_Growth_Equities_Fund_4c2e
Code: ee9cc9a4-cbe0-4c9b-b179-057db13ceca9
Transactions Effective From: 2019-05-20 14:30:27.216636+00:00
Transactions Created On: 2019-05-24 14:

## 11) Value Currency Portfolios

## Value the Currency Forward Contracts

In [26]:
interest_rates = pd.read_csv('data/multiplecurrencies-interestrates.csv')
interest_rates.head()

,currency,interest_rate
0,USD,0.025500
1,AUD,0.017100
2,EUR,-0.001611
3,JPY,0.001000
4,GBP,0.007719


In [27]:
analytics = []
quote_requests = []

for currency, currency_portfolio_code in fund_currencies.items():
    
    if currency!='USD':
        fx_rate = client.quotes.get_quotes(
            scope=scope,
            quote_ids=[
                models.QuoteId(
                    provider='DataScope',
                    price_source='USDRC',
                    instrument_id='{}/USD'.format(currency),
                    instrument_id_type='CurrencyPair',
                    quote_type='Rate',
                    price_side='Mid')
            ],
            effective_at=holdings_effective_date).found[0].metric_value.value
    else:
        fx_rate = 1
    
    response = client.transaction_portfolios.get_transactions(
        scope=scope,
        code=currency_portfolio_code,
        from_transaction_date=holdings_effective_date,
        to_transaction_date=holdings_effective_date,
        filter="type eq 'Buy'")
    
    if len(response.values) == 0:
        continue
        
    hedge_transaction = response.values[0]
    
    hedge_contract = json.loads(client.instruments.get_instrument(
        identifier_type='LusidInstrumentId',
        identifier=hedge_transaction.instrument_uid
    ).instrument_definition.content)
    
    spot_rate = 1 / fx_rate
    
    interest_rate_1 = interest_rates.loc[
        interest_rates['currency'] == 'USD', 'interest_rate'].values[0]
    
    interest_rate_2 = interest_rates.loc[
        interest_rates['currency'] == currency, 'interest_rate'].values[0]
    
    time = (parse(timestr=hedge_contract['settlement_date']) \
            - datetime.now(pytz.UTC)).days / 365
    
    strike_price = hedge_contract['strike_price']

    forward_price = spot_rate * math.exp((interest_rate_2 - interest_rate_1) * time)
    value = (forward_price - strike_price) * math.exp(-interest_rate_2 * time)
    total_value = value * hedge_contract['leg1']['amount']
    
    prettyprint.heading(
        'USD/' + currency+' swap value', '{} {}'.format(
            round(total_value,2), currency))
    
    analytics.append(
        models.InstrumentAnalytic(
            instrument_uid=hedge_transaction.instrument_uid,
            value=total_value
        )
    )
    print (hedge_transaction.instrument_uid)
    quote_requests.append(
        models.UpsertQuoteRequest(
            quote_id=models.QuoteId(
                provider='DataScope',
                price_source='USDRC',
                instrument_id=hedge_transaction.instrument_uid,
                instrument_id_type='LusidInstrumentId',
                quote_type='Price',
                price_side='Mid'),
            metric_value=models.MetricValue(
                value=total_value,
                unit=currency),
            effective_at=holdings_effective_date,
            lineage='InternalSystem'))

response = client.quotes.upsert_quotes(
    scope=scope,
    quotes=quote_requests)

prettyprint.upsert_quotes_response(response)

USD/AUD swap value: -551.11 AUD
LUID_A16AS0XM
USD/GBP swap value: -641.19 GBP
LUID_HYLAGLRF
USD/EUR swap value: -1123.46 EUR
LUID_D2G221EI
USD/JPY swap value: -128597.42 JPY
LUID_42V89JPH
Analytics Set


## Value the Currency Portfolios

In [28]:
currency_portfolio_values = {}

inline_recipe_complex = inline_recipe
inline_recipe_complex.pricing = models.PricingContext(
    options=models.PricingOptions(
        allow_any_instruments_with_sec_uid_to_price_off_lookup=True))

for currency, currency_portfolio_code in fund_currencies.items():
    
    aggregation_request = models.AggregationRequest(
        inline_recipe=inline_recipe_complex,
        effective_at=holdings_effective_date,
        metrics=[
            models.AggregateSpec(key='Holding/default/PV',
            op='Sum'),
            models.AggregateSpec(key='Portfolio/default/Name',
            op='Value'),
            models.AggregateSpec(key='Holding/default/Units',
            op='Value'),
            models.AggregateSpec(key='Instrument/default/Name',
            op='Value'),
            
        ],
        group_by=[
            'Instrument/default/Name'
        ])

    response = client.aggregation.get_aggregation_by_portfolio(
        scope=scope,
        code=currency_portfolio_code,
        request=aggregation_request)
    
    currency_portfolio_values[currency] = response.data[0]['Sum(Holding/default/PV)']
    
    prettyprint.aggregation_response_generic(response)

ApiException: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'Date': 'Fri, 24 May 2019 14:30:45 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'lusid-meta-success': 'False', 'lusid-meta-duration': '920', 'lusid-meta-requestId': '0HLMVM696MPIO:00000001', 'Server': 'FINBOURNE'})
HTTP response body: {"name":"PriceNotFound","errorDetails":[],"code":189,"type":"https://docs.lusid.com/#section/Error-Codes/189","title":"Failed to find or create market data item [Analytic LUID_R99W1V23].","status":404,"detail":"Failed to find or create market data item [Analytic LUID_R99W1V23]. This occurred because LUID_R99W1V23 is not in the loaded Analytics set. Implies incorrect dependencies or missing data for time.","instance":"lusid:api-am-prod.lusid.com/0HLMVM696MPIO/00000001"}


## 12) Securitise Currency Portfolios

In [ ]:
instrument_definitions = {}

for currency in fund_currencies:

    instrument_definitions['currency_{}'.format(currency)] = (models.InstrumentDefinition(
        name=base_portfolio_name,
        identifiers={'ClientInternal': '{}_{}'.format(base_portfolio_name, currency)},
        properties=[models.InstrumentProperty(
            key=circulation_property_key,
            value=models.PropertyValue(
                metric_value=models.MetricValue(
                    value=10000
                    )))],
        look_through_portfolio_id=models.ResourceId(
            scope=scope,
            code='{}_{}'.format(base_portfolio_name, currency)
        )
    ))

response = client.upsert_instruments(
    requests=instrument_definitions
)

prettyprint.instrument_response(response, identifier='ClientInternal')

In [ ]:
instrument_quotes = []

for currency in fund_currencies:

    currency_fund_instrument = client.get_instrument(
        identifier_type='ClientInternal',
        identifier='{}_{}'.format(base_portfolio_name, currency),
        instrument_property_keys=[circulation_property_key])
    
    luid = base_fund_instrument.lusid_instrument_id
    
    currency_fund_total_circulation = round(
        base_fund_instrument.properties[0].value, 0)
    
    currency_fund_unit_price = round(
        currency_portfolio_values[currency] / base_fund_total_circulation, 2)

    instrument_quotes.append(models.UpsertQuoteRequest(
        quote_id=models.QuoteId(
            instrument_id=luid,
            instrument_id_type='LusidInstrumentId',
            quote_type='Price',
            price_side='Mid'),
        metric_value=models.MetricValue(
            value=currency_fund_unit_price,
            unit=currency),
        quote_lineage=models.QuoteLineage(
            data_vendor='Internal',
            contributor='Internal_User'),
        effective_at=holdings_effective_date
    ))
    
    prettyprint.heading(f'{currency} Fund Total Circulation', str(currency_fund_total_circulation))
    prettyprint.heading(f'{currency} Fund Total Value', str(currency_fund_total_value))
    prettyprint.heading(f'{currency} Fund Unit Price', str(currency_fund_unit_price))

response = client.upsert_quotes(
    scope=scope,
    quotes=instrument_quotes)

prettyprint.upsert_quotes_response(response)

## 13) Create Share Class Portfolios

## 14) Value Each Share Class

## 15) Add Transactions to the Base Portfolio

## 16) Value each share class portfolio (bubble up)

## 17) Add Hedging to one of the currency portfolios

## 18) Value each share class portfolio (bubble up)